In [55]:
!pip install pandas
!pip install mysql-connector-python
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 1.7 MB/s eta 0:00:00a 0:00:01


In [1]:
import pandas as pd
import mysql.connector

## CARGAR EL PRIMER DATASET EN STAGING

In [2]:
df = pd.read_csv('../data/raw/ISCED_2013.csv', encoding ='latin-1', delimiter=';')

In [3]:
df.head()

,COD_RAMA,NOM_RAMA,COD_RAMA_N2,NOM_RAMA_N2,COD_RAMA_N3,NOM_RAMA_N3,COD_RAMA_N4,NOM_RAMA_N4,COD_RAMA_N5,NOM_RAMA_N5
0,1,Artes y humanidades,1,Educación,11,Educación,111,Ciencias de la educación,11101,Pedagogía
1,1,Artes y humanidades,1,Educación,11,Educación,111,Ciencias de la educación,11102,Didácticas específicas
2,1,Artes y humanidades,1,Educación,11,Educación,112,Formación de docentes de enseñanza infantil,11201,Educación infantil
3,1,Artes y humanidades,1,Educación,11,Educación,113,Formación de docentes de enseñanza primaria,11301,Educación primaria
4,1,Artes y humanidades,1,Educación,11,Educación,114,Formación de docentes de educación secundaria ...,11401,Otros maestros


In [ ]:
columns = df.columns
new_columns_dict = {}
cod_count = 1
nom_count = 1

for col in columns:
    if 'COD_' in col:
        new_columns_dict[col] = f'codigo_rama_{cod_count}'
        cod_count += 1
    elif 'NOM_' in col:
        new_columns_dict[col] = f'nombre_rama_{nom_count}'
        nom_count += 1
    else:
        new_columns_dict[col] = col

# Use the rename() method to rename the columns
df.rename(columns=new_columns_dict, inplace=True)

# Print the updated DataFrame
df.head(2)

In [48]:
cnx =mysql.connector.connect(
    host='localhost',
    user='admin',
    password='admin',
    database='dw',
    auth_plugin='mysql_native_password'
)

cursor = cnx.cursor()

In [ ]:
data = [tuple(row) for _, row in df.iterrows()]

query=f"INSERT INTO stage_ramas_conocimiento VALUES ({','.join(['%s'] * len(df.columns))})"

cursor.executemany(query, data)

try:
    cnx.commit()
    print("Datos insertados de manera exitosa!")
except Exception as e:
    print("Error al insertar los datos")
    print("Error code:", e.errno)
    print("Error description:", e.strerror)

In [ ]:
query=f"SELECT * FROM stage_ramas_conocimiento"

df_2 = pd.read_sql(query, cnx)

df_2.head(2)

cursor.close()

In [ ]:
#mysql -u admin -p

## CARGAR EL SEGUNDO DATASET EN STAGING

In [4]:
#Read data from SEGR1.csv SEGR2.csv files
df_1 = pd.read_csv('../data/raw/SEGR1.csv', encoding ='latin-1', delimiter=';')
df_2 = pd.read_csv('../data/raw/SEGR2.csv', encoding ='latin-1', delimiter=';')

In [16]:
df_union = pd.concat([df_1, df_2])
df_union.head(2)

,TIPO_UNIVERSIDAD,MODALIDAD,UNIVERSIDAD,RAMA_ENSEÑANZA,EGR_C16_17,EGR_C15_16,EGR_C14_15,EGR_C13_14,EGR_C12_13,EGR_C11_12,EGR_C10_11,EGR_C09_10
0,Universidades Privadas,Presencial,Abat Oliba CEU,Ciencias Sociales y Jurídicas,450,370,485,333,262,356,422,246
1,Universidades Privadas,Presencial,Abat Oliba CEU,Ingeniería y Arquitectura,0,0,0,0,0,0,0,0


In [37]:
data_cols = df_union.columns

id_columns = [col for col in data_cols if not 'EGR_' in col]
data_columns = [col for col in data_cols if 'EGR_' in col]

In [43]:
#Consolidar información de todos los años en una sola columna
df_unpivot = pd.melt(df_union, id_vars=id_columns, value_vars=data_columns, var_name='año', value_name='num_egresados')

#Actualizar columna de año usando los últimos dos digitos del encabezado
df_unpivot['año'] = '20' + df_unpivot['año'].str[-2:]

df_unpivot['pais'] = 'España'

In [44]:
#Validar resultados de la columna año
df_unpivot['año'].value_counts()

año
2017    410
2016    410
2015    410
2014    410
2013    410
2012    410
2011    410
2010    410
Name: count, dtype: int64

In [45]:
df_unpivot.rename(columns={'UNIVERSIDAD':'nombre_universidad', 'TIPO_UNIVERSIDAD': 'tipo_universidad', 'MODALIDAD': 'modalidad', 'RAMA_ENSEÑANZA':'rama_enseñanza'}, inplace=True)

In [46]:
df_unpivot = df_unpivot[['año','pais','nombre_universidad','tipo_universidad','modalidad','rama_enseñanza','num_egresados']]
df_unpivot.head(2)

,año,pais,nombre_universidad,tipo_universidad,modalidad,rama_enseñanza,num_egresados
0,2017,España,Abat Oliba CEU,Universidades Privadas,Presencial,Ciencias Sociales y Jurídicas,450
1,2017,España,Abat Oliba CEU,Universidades Privadas,Presencial,Ingeniería y Arquitectura,0


In [49]:
cursor = cnx.cursor()

data = [tuple(row) for _, row in df_unpivot.iterrows()]

query=f"INSERT INTO stage_egresados_universidad VALUES ({','.join(['%s'] * len(df_unpivot.columns))})"

cursor.executemany(query, data)

try:
    cnx.commit()
    print("Datos insertados de manera exitosa!")
except Exception as e:
    print("Error al insertar los datos")
    print("Error code:", e.errno)
    print("Error description:", e.strerror)

Datos insertados de manera exitosa!


In [52]:
query=f"SELECT * FROM stage_egresados_universidad"

df_2 = pd.read_sql(query, cnx)

df_2.head(2)

/tmp/ipykernel_285/5771051.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_2 = pd.read_sql(query, cnx)


,ao,pais,nombre_universidad,tipo_universidad,modalidad,rama_enseanza,num_egresados
0,2017,España,Abat Oliba CEU,Universidades Privadas,Presencial,Ciencias Sociales y Jurídicas,450
1,2017,España,Abat Oliba CEU,Universidades Privadas,Presencial,Ingeniería y Arquitectura,0


In [ ]:
cursor.close()

## CARGAR EL TERCER DATASET EN STAGING

In [96]:
df_1 = pd.read_excel('../data/raw/educ_uoe_grad01.xlsx', sheet_name='Data', skiprows=11, nrows=13)
df_2 = pd.read_excel('../data/raw/educ_uoe_grad01.xlsx', sheet_name='Data2', skiprows=11, nrows=13)
df_3 = pd.read_excel('../data/raw/educ_uoe_grad01.xlsx', sheet_name='Data3', skiprows=11, nrows=13)

In [97]:
df_1.drop('Unnamed: 6', inplace=True, axis=1)

In [98]:
#Consolidar información de todos los años en una sola columna
df1_unpivot = pd.melt(df_1, id_vars=['GEO/TIME'], value_vars=['2013','2014','2015','2016','2017'], var_name='year', value_name='num_graduated')

#Consolidar información de todos los años en una sola columna
df2_unpivot = pd.melt(df_2, id_vars=['GEO/TIME'], value_vars=['2013','2014','2015','2016','2017'], var_name='year', value_name='num_graduated_M')

#Consolidar información de todos los años en una sola columna
df3_unpivot = pd.melt(df_3, id_vars=['GEO/TIME'], value_vars=['2013','2014','2015','2016','2017'], var_name='year', value_name='num_graduated_F')

In [99]:
merge_df = df1_unpivot.merge(df2_unpivot, on=['GEO/TIME','year'] , how='left')
merge_df = merge_df.merge(df3_unpivot, on=['GEO/TIME','year'], how='left')
merge_df.head(2)

,GEO/TIME,year,num_graduated,num_graduated_M,num_graduated_F
0,Denmark,2013,66467,28254,38213
1,Germany (until 1990 former territory of the FRG),2013,495808,244570,251238


In [101]:
merge_df.rename(columns={'GEO/TIME': 'country'}, inplace=True)

#Cambiar el orden de las columnas
merge_df = merge_df[['year','country','num_graduated_M','num_graduated_F','num_graduated']]

merge_df.replace(':', 0, inplace=True)

In [102]:
cursor = cnx.cursor()

data = [tuple(row) for _, row in merge_df.iterrows()]

query=f"INSERT INTO stage_numero_egresados_internacional VALUES ({','.join(['%s'] * len(merge_df.columns))})"

cursor.executemany(query, data)

try:
    cnx.commit()
    print("Datos insertados de manera exitosa!")
except Exception as e:
    print("Error al insertar los datos")
    print("Error code:", e.errno)
    print("Error description:", e.strerror)

Datos insertados de manera exitosa!


In [105]:
query=f"SELECT * FROM stage_numero_egresados_internacional"

df_2 = pd.read_sql(query, cnx)

df_2.head(2)

/tmp/ipykernel_285/3675124467.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_2 = pd.read_sql(query, cnx)


,year,country,num_graduated_M,num_graduated_F,num_graduated
0,2013,Denmark,28254,38213,66467
1,2013,Germany (until 1990 former territory of the FRG),244570,251238,495808


In [106]:
cursor.close()

True

In [ ]:
cnx.close()